In [ ]:
import keras
print(keras.__version__)
import numpy as np 
import pandas as pd 
import tensorflow as tf
print(tf.__version__)
import zipfile
from matplotlib import pyplot as plt
from tqdm import tqdm
%matplotlib inline
import sys
import datetime
import time
import nltk, re, string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from keras.models import Model
from keras.layers import Input, Dense, Embedding, concatenate
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D
from keras.layers import LSTM, CuDNNGRU, Add, BatchNormalization, Activation, CuDNNLSTM, Dropout
from keras.layers import *
from keras.models import *
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import gc
from sklearn import metrics
# XGboost related
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from scipy.sparse import csr_matrix, hstack

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

In [ ]:
train_df = pd.read_csv('train.csv',sep='\t')
#train_df["question_text"] = train_df["question_text"].map(lambda x: clean_text(x))

test_df = pd.read_csv('test.csv',sep='\t')
#test_df["question_text"] = test_df["question_text"].map(lambda x: clean_text(x))

# train_ques_lens = train_df['question_text'].map(lambda x: len(x.split(' ')))
# test_ques_lens = test_df['question_text'].map(lambda x: len(x.split(' ')))
# print('Train text max len:', train_ques_lens.max())
# print('Test text max len:', test_ques_lens.max())
# plt.figure(figsize=(10, 4))
# sns.kdeplot(train_ques_lens)
# sns.kdeplot(test_ques_lens)
# plt.legend(('train', 'test'))
# plt.show()
# del train_ques_lens; del test_ques_lens
# gc.collect()
# pass

In [ ]:
maxlen = 70
max_features = 90000  # capsule 95000
embed_size = 300

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", 
                 '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', 
                 '³': '3', 'π': 'pi', }

def clean_text(x):
    x = str(x)
    for p in punct_mapping:
        x = x.replace(p, punct_mapping[p])
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", 
                "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": 
                "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
                "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
                "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", 
                "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", 
                "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [ ]:
start =time.clock()
# lower
train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
test_df["question_text"] = test_df["question_text"].apply(lambda x: x.lower())

# Clean the text
train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))

# Clean numbers
train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_numbers(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_numbers(x))

# Clean speelings
train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: replace_typical_misspell(x))

X_train = train_df["question_text"].fillna("na").values  

X_test = test_df["question_text"].fillna("na").values
y = train_df["target"]
end = time.clock()
print('Running time: %s Seconds'%(end-start))

In [ ]:
X_train_word_list = []
X_test_word_list = []
for i in range(len(X_train)):
    X_train_word_list.append(X_train[i].split(' '))
for i in range(len(X_test)):
    X_test_word_list.append(X_test[i].split(' '))

In [ ]:
from gensim import corpora
# 赋给语料库中每个词(不重复的词)一个整数id
word_list = X_train_word_list+X_test_word_list
dictionary = corpora.Dictionary(word_list)
corpus = [dictionary.doc2bow(text) for text in word_list]
# [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]
# 元组中第一个元素是词语在词典对应的id，第二个元素是词语在句子中出现的次数
del word_list; gc.collect() 

# 训练模型并保存
from gensim import models
tfidf = models.TfidfModel(corpus)
# tfidf.save("my_model.tfidf")
# # 载入模型
# tfidf = models.TfidfModel.load("my_model.tfidf")

In [ ]:
start = time.clock()
X_train_id = []  # 将句子表示成单词在词典中id的形式
# [[6, 5, 11, 9, 12, 14, 10, 4, 3, 8, 7, 13, 1, 2, 0], [20, 29, 23, 18, 17, 21, 15, 0, 6, 28, 29, 22, 25, 27, 16, 19, 24, 26, 2, 0], [36, 31, 35, 30, 34, 2, 0, 31, 35, 30, 33, 32, 2, 0]]
X_test_id = []
word_id_dict = dictionary.token2id
for i in range(len(X_train_word_list)):
    sen_id = []
    word_sen = X_train_word_list[i]
    for j in range(len(word_sen)):       
        id = word_id_dict.get(word_sen[j])
        if id is None:
            id = 0 
        sen_id.append(id)
    X_train_id.append(sen_id)

for i in range(len(X_test_word_list)):
    sen_id = []
    word_sen = X_test_word_list[i]
    for j in range(len(word_sen)):       
        id = word_id_dict.get(word_sen[j])
        if id is None:
            id = 0 
        sen_id.append(id)
    X_test_id.append(sen_id)
end = time.clock()
print('Running time: %s Minutes'%((end-start)/60))

In [ ]:
start = time.clock()
X_train_tfidf_vec = []  # 每个句子是一个字典，key是单词的ID，value是单词对应的tfidf值
for i in range(len(X_train)):
    temp = {}
    string = X_train[i]
    string_bow = dictionary.doc2bow(string.lower().split()) 
    string_tfidf = tfidf[string_bow]
    # 每个句子是一个list，句中的每个单词表示为一个元组，元组的第一个元素是单词的ID，第二个元素是tfidf值
    for j in range(len(string_tfidf)):
#         print(string_tfidf[j][0])
        temp[string_tfidf[j][0]] = string_tfidf[j][1]
#         print(temp)
    X_train_tfidf_vec.append(temp)
# print(X_train_tfidf_vec)
print(len(X_train_tfidf_vec))

X_test_tfidf_vec = []  # 每个句子是一个字典，key是单词的ID，value是单词对应的tfidf值
for i in range(len(X_test)):
    temp = {}
    string = X_test[i]
    string_bow = dictionary.doc2bow(string.lower().split()) 
    string_tfidf = tfidf[string_bow]
    # 每个句子是一个list，句中的每个单词表示为一个元组，元组的第一个元素是单词的ID，第二个元素是tfidf值
    for j in range(len(string_tfidf)):
#         print(string_tfidf[j][0])
        temp[string_tfidf[j][0]] = string_tfidf[j][1]
#         print(temp)
    X_test_tfidf_vec.append(temp)
print(len(X_test_tfidf_vec))
    
end = time.clock()
print('Running time: %s Seconds'%(end-start))  # 119.35793299999978 Seconds

In [ ]:
X_train_tfidf = []  # tfidf值形成的句子。每个句子是一个list
for i in range(len(X_train_id)):
    sen_id = X_train_id[i]
    sen_id_tfidf = X_train_tfidf_vec[i]
    sen = []
    for j in range(len(sen_id)):
        word_id = sen_id[j]
        word_tfidf = sen_id_tfidf.get(word_id)
        if word_tfidf is None:
            word_tfidf = 0
        sen.append(word_tfidf)
    X_train_tfidf.append(sen)

X_test_tfidf = []  # tfidf值形成的句子。每个句子是一个list
for i in range(len(X_test_id)):
    sen_id = X_test_id[i]
    sen_id_tfidf = X_test_tfidf_vec[i]
    sen = []
    for j in range(len(sen_id)):
        word_id = sen_id[j]
        word_tfidf = sen_id_tfidf.get(word_id)
        if word_tfidf is None:
            word_tfidf = 0
        sen.append(word_tfidf)
    X_test_tfidf.append(sen)

In [ ]:
x_train_tfidf = sequence.pad_sequences(X_train_tfidf, maxlen=maxlen,dtype='float64') # (1306122, 70)
x_test_tfidf = sequence.pad_sequences(X_test_tfidf, maxlen=maxlen,dtype='float64') # (56370, 70)

In [ ]:
del X_train_word_list; gc.collect() 
del X_test_word_list; gc.collect() 
del X_train_id; gc.collect() 
del X_test_id; gc.collect() 
del X_train_tfidf_vec; gc.collect() 
del X_test_tfidf_vec; gc.collect() 
del X_train_tfidf; gc.collect() 
del X_test_tfidf; gc.collect() 
del tfidf; gc.collect() 
del corpus; gc.collect() 
del dictionary; gc.collect() 
del word_id_dict; gc.collect() 

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))

X_train_emb = tokenizer.texts_to_sequences(X_train)
X_test_emb = tokenizer.texts_to_sequences(X_test)

x_train_emb = sequence.pad_sequences(X_train_emb, maxlen=maxlen) # (1306122, 70)
x_test_emb = sequence.pad_sequences(X_test_emb, maxlen=maxlen) # (56370, 70)

In [ ]:
del X_train_emb; gc.collect() 
del X_test_emb; gc.collect() 

In [ ]:
print(x_train_emb.shape)
print(x_test_emb.shape)
print(x_train_emb.astype)

In [ ]:
x_train_input = np.concatenate((x_train_emb, x_train_tfidf), axis=1)
x_test_input = np.concatenate((x_test_emb, x_test_tfidf), axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_tra, X_val, y_tra, y_val = train_test_split(x_train_input, y, test_size = 0.1, random_state=42)

In [ ]:
print(x_test_input[0][139:140])
print(x_train_emb[0][68:70])
print(x_train_tfidf[0][0:2])
print(x_train_input.shape)


In [ ]:
del x_train_emb; gc.collect() 
del x_train_tfidf; gc.collect() 
del x_test_emb; gc.collect() 
del x_test_tfidf; gc.collect() 
del x_train_input; gc.collect() 

In [ ]:
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    TIME_STEPS = inputs.shape[1].value
    SINGLE_ATTENTION_VECTOR = False
    
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [ ]:
# EMBEDDING_FILE = '/Users/zhaofangyu/Downloads/glove.6B/glove.6B.300d.txt'
# def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
# embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
# print("embedding_1 !!")
# all_embs = np.stack(embeddings_index.values())
# emb_mean,emb_std = all_embs.mean(), all_embs.std()
# embed_size = all_embs.shape[1]

# word_index = tokenizer.word_index
# nb_words = min(max_features, len(word_index))
# embedding_matrix_1 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
# for word, i in word_index.items():
#     if i >= max_features: continue
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: embedding_matrix_1[i] = embedding_vector

# del embeddings_index; gc.collect() 

In [ ]:
# EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
# def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
# embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)
# print("embedding_3 !!")
# all_embs = np.stack(embeddings_index.values())
# emb_mean,emb_std = all_embs.mean(), all_embs.std()
# embed_size = all_embs.shape[1]

# word_index = tokenizer.word_index
# nb_words = min(max_features, len(word_index))
# embedding_matrix_3 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
# for word, i in word_index.items():
#     if i >= max_features: continue
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: embedding_matrix_3[i] = embedding_vector
        
# del embeddings_index; gc.collect()   

In [ ]:
# embedding_matrix = np.mean((embedding_matrix_1), axis=0)# (90000, 300)
# print(embedding_matrix.shape)
# del embedding_matrix_1; gc.collect()

In [ ]:
def row_slice(x, start, end):
    return x[:, start:end]
def change_type(x):
    return tf.to_int32(x, name='ToInt32')
def matmul(x, y):
    return tf.matmul(x, y)
def concat(x, y, axis):
    return tf.concat([x, y], axis=axis)

In [ ]:
def model2():
    inp = Input(shape=(maxlen*2, ))
    embed = Embedding(max_features, embed_size * 1, trainable=False)(inp)
    x = embed
    x = Bidirectional(LSTM(128, unroll=True,return_sequences=True))(x)
    x = attention_3d_block(x)
    x = Bidirectional(LSTM(128, unroll=True,return_sequences=True))(x)
    x = AttLayer(64)(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    outp = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 
    return model

In [ ]:
def model2():
    inp = Input(shape=(maxlen*2, ))  # (?, 140)
    print("inp",inp.shape)
    inp_emb = Lambda(row_slice, arguments={'start':0, 'end':70})(inp)  # (?, 70)  <dtype: 'float32'>
    inp_emb = Lambda(change_type)(inp_emb)  # tensor <dtype: 'int32'>

    inp_tfidf = Lambda(row_slice, arguments={'start':70, 'end':140})(inp)  # # (?, 70)
    inp_tfidf = Reshape((maxlen, 1))(inp_tfidf)  # tensor (?, 70, 1)

    
#     embed = Embedding(max_features, embed_size * 1, weights=[embedding_matrix], trainable=False)(inp_emb)
    embed = Embedding(max_features, embed_size * 1, trainable=False)(inp_emb)
    # (?, 70, 300)
    embed_trans = Permute((2, 1))(embed)  # (?, 300, 70)
    # arguments={'x':embed_trans, 'y':inp_tfidf}
    sen_emb = Lambda(matmul, arguments={'y':inp_tfidf})(embed_trans)   # (?, 300, 1)
    emb_mul_tfidf = Lambda(matmul, arguments={'y': Permute((2, 1))(sen_emb)})(inp_tfidf)   # (?, 70, 300)
    
    x = Lambda(concat, arguments={'y':emb_mul_tfidf, 'axis':2})(embed)
#     x = tf.concat([embed, emb_mul_tfidf], axis=2)  # (?, 70, 600)
    print(x.shape)
    
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    print("x shape: ", x.shape)
    x = BatchNormalization()(x)
    print("x shape: ", x.shape)  # (?, 70, 256)
    x = attention_3d_block(x)
    print("x shape: ", x.shape)
    x = BatchNormalization()(x)
    print("x shape after attention: ", x.shape)  # (?, 70, 256)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = BatchNormalization()(x)
    print("x shape: ", x.shape)  # (?, 70, 256)
    x = AttLayer(64)(x)
    x = BatchNormalization()(x)
    print("x shape after attention: ", x.shape)  # (?, 256)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    outp = Dense(1, activation="sigmoid")(x)
    outp = Dense(1, activation="sigmoid")(x)
    print("outp:",outp.shape)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])    

    return model

In [ ]:
# print(X_tra.shape)
MODEL2 = model2()
MODEL2.summary()

batch_size = 512
epochs = 4

early_stopping = EarlyStopping(patience=3, verbose=1, monitor='val_loss', mode='min')
model_checkpoint = ModelCheckpoint('./model2.model', save_best_only=True, verbose=1, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3, min_lr=0.0001, verbose=1)

hist = MODEL2.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
# MODEL2.save('./model2.h5')
print("finished!!")

In [ ]:
pred_val_y_2 = MODEL2.predict([X_val], batch_size=1024, verbose=1)
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    acc = metrics.accuracy_score(y_val, (pred_val_y_2 > thresh).astype(int))
    f1_score = metrics.f1_score(y_val, (pred_val_y_2 > thresh).astype(int))
    pre = metrics.precision_score(y_val, (pred_val_y_2 > thresh).astype(int))
    recall = metrics.recall_score(y_val, (pred_val_y_2 > thresh).astype(int))
    thresholds.append([thresh, f1_score])
    print("acc score at threshold {0} is {1}".format(thresh, acc))
    print("F1 score at threshold {0} is {1}".format(thresh, f1_score))
    print("precision at threshold {0} is {1}".format(thresh, pre))
    print("recall at threshold {0} is {1}".format(thresh, recall))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
best_thresh_f1 = thresholds[0][1]
print("Best threshold:{0} ， f1:{1}".format(str(best_thresh), str(best_thresh_f1)))
# print("Best threshold:{0} ， f1:{1}".format(best_thresh_2, best_thresh_2_f1))

# metrics.roc_curve(pred_val_y_2, y_val, pos_label=None, sample_weight=None, drop_intermediate=True)

y_pred_2 = MODEL2.predict(x_test_input, batch_size=1024, verbose=True)

In [ ]:
y_pred = y_pred_2
y_te = (y_pred[:,0] > best_thresh).astype(np.int)

submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)